In [16]:
from scipy import signal 
import numpy as np
import timeit

import arlpy.uwapm as pm
import arlpy.plot as aplt

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from bokeh.models.annotations import Title
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.palettes import Category10,viridis,Spectral11
from bokeh.models import ColumnDataSource, Range1d, BoxAnnotation, HoverTool, LogColorMapper, LogTicker, ColorBar
output_notebook()

samplefreq = 100*10**3 #Hz

def impulse_response(h,r,d1,d2,samplefreq, fc, **kwargs):
    Type = kwargs.get('Type', 'valid')

    ssp = 1520
    
    surface = np.array([[r, 0.5+0.5*np.sin(2*np.pi*0.05*r)] for r in np.linspace(0,1000,1001)])
    maxAngle = 180*(np.arctan(h/(r/2)))/np.pi 
    env = pm.create_env2d(depth = h, rx_range = r, tx_depth = d1, rx_depth = d2, frequency = fc, max_angle = maxAngle, min_angle = -maxAngle, surface = surface, soundspeed = ssp)
    arrivals = pm.compute_arrivals(env)
    Hn = pm.arrivals_to_impulse_response(arrivals,samplefreq)
    
    if Type == 'valid':
        return Hn.real
    if Type == 'full':
        return env, arrivals, Hn.real
    
def CorrDiversity(h,r,d1,d21,d22_list,samplefreq, fc, bw, **kwargs):
    Type = kwargs.get('Type', 'multi')
    
    lowfreq = fc-(bw/2)
    highfreq = fc+(bw/2)
    
    CMNL = []
    
    Hn1 = impulse_response(h,r,d1,d21,samplefreq,fc)
    signalSize1 = round(len(Hn1))
    freq_l = np.fft.fftfreq(signalSize1,1000/samplefreq)
    max_new_list1 = np.abs(np.fft.fft(Hn1)) #freq_list == samplefreq
    count_low, count_high = 0,0
    for i in range(len(freq_l)):
        if freq_l[i] <= lowfreq and freq_l[i] >=0:
            count_low += 1
        if freq_l[i] >= highfreq or freq_l[i] <=0:
            count_high += 1

    max_new_list1 = max_new_list1[count_low:len(max_new_list1)-count_high]
    freq_list1 = freq_l[count_low:len(freq_l)-count_high]
   
    mean1 = np.mean(max_new_list1)
    A = [max_value-mean1 for max_value in max_new_list1]
    
    if Type == 'multi':
        for d22 in d22_list:
            Hn2 = impulse_response(h,r,d1,d22,samplefreq,fc)
            signalSize2 = round(len(Hn2))
            freq_l = np.fft.fftfreq(signalSize2,1000/samplefreq)
            max_new_list2 = np.abs(np.fft.fft(Hn2))
            count_low, count_high = 0,0
            for i in range(len(freq_l)):
                if freq_l[i] <= lowfreq and freq_l[i] >=0:
                    count_low += 1
                if freq_l[i] >= highfreq or freq_l[i] <=0:
                    count_high += 1
            
            max_new_list2 = max_new_list2[count_low:len(max_new_list2)-count_high]
            freq_list2 = freq_l[count_low:len(freq_l)-count_high]
            
            mean2 = np.mean(max_new_list2)
            B = [max_value-mean2 for max_value in max_new_list2]
            
            CMNL.append(sum(a*b for a, b in zip(A, B))/(np.sqrt((sum(a**2 for a in A)*sum(b**2 for b in B)))))
            normalizedCMNL = [CMNV/max(CMNL) for CMNV in CMNL]

        return normalizedCMNL       
    
    if Type == 'single':        
        Hn2 = impulse_response(h,r,d1,d22_list,samplefreq,fc)
        signalSize2 = round(len(Hn2))
        freq_l = np.fft.fftfreq(signalSize2,1000/samplefreq)
        max_new_list2 = np.abs(np.fft.fft(Hn2))
        count_low, count_high = 0,0
        for i in range(len(freq_l)):
            if freq_l[i] <= lowfreq and freq_l[i] >=0:
                count_low += 1
            if freq_l[i] >= highfreq or freq_l[i] <=0:
                count_high += 1
        
        max_new_list2 = max_new_list2[count_low:len(max_new_list2)-count_high]
        freq_list2 = freq_l[count_low:len(freq_l)-count_high]
        mean2 = np.mean(max_new_list2)
        B = [max_value-mean2 for max_value in max_new_list2]

        CMNV = (sum(a*b for a, b in zip(A, B))/(np.sqrt((sum(a**2 for a in A)*sum(b**2 for b in B))))) 
    
        toc=timeit.default_timer()
        #print("time taken: ", toc-tic, 's') 
        return CMNV
    
def avgXCorrForDeltas(H, R, D1, D2min, D2max, deltaMin, deltaMax, thres, fc, bw, **kwargs):
    length = kwargs.get('Len', 'valid')
    
    delta_list = np.linspace(deltaMin,deltaMax,80)#(deltaMax-deltaMin)*25)
    d21_list = np.linspace(D2min,D2max,50) #for averaging purpose
    
    multiAvgCMNV = []
    
    if length == 'full':
        for j in range(len(delta_list)):
            forAverageCMNV_list = []
            for i in range(len(d21_list)):
                d22 = 0
                d22 = d21_list[i]+delta_list[j]
                CMNV = CorrDiversity(H,R,D1,d21_list[i],d22,samplefreq,fc,bw,Type = 'single')
                forAverageCMNV_list.append(CMNV)
            avgCMNV = np.mean(forAverageCMNV_list)
            multiAvgCMNV.append(avgCMNV)
            normMultiAvgCMNV = [value/max(multiAvgCMNV) for value in multiAvgCMNV]
            if normMultiAvgCMNV[j] > thres:
                imptDelta = delta_list[j]
    
    if length == 'valid':
        for j in range(len(delta_list)):
            forAverageCMNV_list = []
            for i in range(len(d21_list)):
                d22 = 0
                d22 = d21_list[i]+delta_list[j]
                CMNV = CorrDiversity(H,R,D1,d21_list[i],d22,samplefreq,fc,bw,Type = 'single')
                forAverageCMNV_list.append(CMNV)
            avgCMNV = np.mean(forAverageCMNV_list)
            multiAvgCMNV.append(avgCMNV)
            normMultiAvgCMNV = [value for value in multiAvgCMNV]
            if normMultiAvgCMNV[j] < thres:
                imptDelta = delta_list[j]
                break
            
        delta_list = delta_list[:len(normMultiAvgCMNV)]
    
    return delta_list, normMultiAvgCMNV, imptDelta

Loading BokehJS ...

In [ ]:
H,R,D1 = 300,100, 5
D2min, D2max = 5,7
deltaMin, deltaMax = 0, 1
thres = 0.1
fc, bw = 20,10

tic = timeit.default_timer()
delta_list, CMNV, delta = avgXCorrForDeltas(H, R, D1, D2min, D2max, deltaMin, deltaMax, thres, fc, bw, Len = 'full')
toc = timeit.default_timer()

FI = figure(title="average xCorrelartion for different deltas", x_axis_label = "Delta (m)", y_axis_label = "xcorr values",plot_width=950, plot_height=500, logo = None)
bottom, top = -1, 1
FI.y_range=Range1d(bottom, top)
FI.line(delta_list,CMNV)
FI.ray(x=[deltaMax], y=[thres], length=deltaMax, angle=np.pi, line_width=1, line_color = '#000000')
toc=timeit.default_timer()
print("time taken: ", toc-tic, 's')
print("delta point:", delta)
show(FI)